In [18]:
import gensim
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim import corpora, models
import numpy as np
import pandas as pd
from collections import OrderedDict

In [6]:
def loadAndProcess(filename): 
    f=open(filename)
    text=f.read()
    sents=sent_tokenize(text) #break the document into smaller sentences
    
    #remove sentences that have less than 5 words in total
    i=0
    while(i<len(sents)):
        sent=nltk.word_tokenize(sents[i])
        if len(sent)<5:
            del sents[i]
        i=i+1
        
    #remove stop words
    for i in range(len(sents)):
        words=nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(sents[i])
        words=[w.lower() for w in words if w not in stopwords.words('english')]
        words=" ".join(words)
        sents[i]=words
        
    return(sents)

In [10]:
def loadOnly(filename):
    f=open(filename)
    text=f.read()
    sents=sent_tokenize(text) #break the document into smaller sentences
    
    #remove sentences that have less than 5 words in total
    i=0
    while(i<len(sents)):
        sent=nltk.word_tokenize(sents[i])
        if len(sent)<5:
            del sents[i]
        i=i+1
    return(sents)        

In [7]:
def prepareCorpus(sentences):
    text=list()
    for i in range(len(sentences)):
        words=nltk.word_tokenize(sentences[i])
        text.append(words)
    dictionary = corpora.Dictionary(text)
    corpus = [dictionary.doc2bow(t) for t in text]
    return(dictionary,corpus)

In [12]:
original=loadOnly("data")  #original sentences without any pre processing
sents=loadAndProcess("data") #processed sentences
dic,corp=prepareCorpus(sents) #dictionary and corpus
numOfTopics=2           #change as per need
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corp, num_topics=numOfTopics, id2word = dic, passes=20)

In [14]:
docsTopicMatrix=np.zeros((len(sents),numOfTopics))
for i in range(len(sents)):
    topicsList=ldamodel.get_document_topics(corp[i],minimum_probability=0)
    for j in range(len(topicsList)):
        docsTopicMatrix[i,j]=topicsList[j][1]  

In [15]:
docsTopicMatrix=pd.DataFrame(docsTopicMatrix)